# Mean reversion strategy

1. Johanson test for a portfolio of stocks 

2. Based on eigenvector from the test, construct the portfolio

3. Normalize the eigenvector to weights, we can normalize such that all positive weights sum up to 1 (or negative weights sum up to 1)
$$\hat v = v / \sum_i (v_i > 0) * v_i$$


3. determine the half life of the mean reversion

4. trade the portfolios (either based on Z score or whatever)

<b>Some of my reminders</b>:

- price conintegration means there is a constnat combination that x shares of stock i + y shares of stock j = constant => no rebalance is needed

- log price cointegration means there is a constnat combniation that x * return of stock i + y * return of stock j = constant => this will require daily rebalance


<b>How to interpret the Johanson test reesults</b>

- res = coint_johansen(np.log(close[['GDX', 'GLD']]), det_order=0, k_ar_diff=1)

- res.lr1 is the trace test statistics, res.lr2 is the max eigenvalue statistics. In general, if either lr1 or lr2 exceeds the critical value, it indicates evidence to reject the null hypothesis

- res.cvm is the critical value of res.lr2 at 90%/95%/99% (first column arr = 90%, second column arr = 95%......)

- res.cvt is the critical value of res.lr1 at 90%/95%/99% (first column arr = 90%, second column arr = 95%......)

- the null hypothesis is testing the rank of $\Delta$ (e.g. r<=0, r<=1 ... r<=n-1), this indicates how many cointegration relationship are there

- res.evec[:,0] represents the eigenvector, sorted from highest eigenvalues


# Predefined Paris

We start with some well-known contingration pairs from either reference book or internet. Each pairs here shuold have a reason to justify why they are cointegrated

ETFs Pairs - reference book suggests that ETF pairs are likely to cointegrate out of sample because ETF each contains basket of stocks rather than single company

- One way to identify ETF pairs are <b>commodity ETF and ETF of companies that produce that commodity</b>

    - GLD / GDX / USO
    
        - GLD are gold price ETF, GDX are gold miner ETF, USO tracks price of oil futures. Empirical support shown that adding USO to portfolio of GDX/GLD has significantly improvve the Johanson Test
        </br></br>

    - USO / XLE

        - USO tracks crude oil future prices, XLE tracks the performance of Energy Select Sector Index
        </br></br>

    -  EWA / EWC

In [3]:
%load_ext autoreload
%autoreload 2

from account.Futu import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
from datetime import datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly
import warnings
from utils.data_helper import *
from utils.etf_lists import *
from utils.logging import get_logger
import yfinance as yf
from tqdm import tqdm
from itertools import cycle
from strategy_v1 import InterdayPairsMeanReversion, InterdayMeanReversion
from itertools import combinations

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 50
pd.options.display.max_rows = 200

logger = get_logger('Mean Reversion Strategy')

# BackTest for one pair

1. Given a portfolios

2. Define a rolling windows which we calculate the eigenvector and construct a cointegrated portfolios

3. trade the spreads when Zscore > threshold

4. If z score is further above a threshold, this might imply your portfolio is no longer cointegrated anymore, repeat step 2 and 3

# My Notes

1. More frequent rebalance once found the rolling series are not cointegrated generallly enhanced performance


# Next Step
1. Dynamic attribute the capital across different portfolios

2. AAAU and OUNZ looks like a good exmaple to trade

In [6]:
start_date = datetime(2021,1,1)
end_date = get_today(-1)

symbols = ['GLD', 'GDX', 'USO']
symbols = ['UCO', 'UGA']
symbols = ['OILK','SCO']
symbols = ['BOIL', 'SCO']
symbols = ['SCO', 'UNL']
symbols = ['DBO', 'UCO']
symbols = ['AGQ', 'ZSL']
symbols = ['DBC','FTGC']
symbols = ['AAAU','OUNZ']
symbols = ['DJCB', 'DJP']
symbols = ['GCC', 'SDCI']
symbols = ['PDBC','UCIB']
symbols = ['FTGC', 'SDCI']
#symbols = ['PPLT', 'ZSL']
#symbols = ['EWA', 'EWC']
#symbols = ['TSLA', 'AAPL']

In [7]:
strategy = InterdayPairsMeanReversion()
strategy.set_stock_universe(symbols)
strategy.set_start_date(start_date)
strategy.set_end_date(end_date)
strategy.preprocess_data()
strategy.generate_position()
strategy.backtest_summary(show_rebal=True)

[*********************100%***********************]  2 of 2 completed

2024-07-30 02:45:21,471 - Interday Pairs Mean Reversion - INFO - Rebalance portfolio based on Johanson Test at rank 1: 2022-01-03
2024-07-30 02:45:21,502 - Interday Pairs Mean Reversion - INFO - Rebalance portfolio based on Johanson Test at rank 1: 2022-03-01
2024-07-30 02:45:21,517 - Interday Pairs Mean Reversion - INFO - Rebalance portfolio based on Johanson Test at rank 1: 2022-03-21
2024-07-30 02:45:21,564 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2022-06-22
2024-07-30 02:45:21,572 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2022-07-07
2024-07-30 02:45:21,581 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2022-07-21
2024-07-30 02:45:21,590 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2022-08-04
20

2024-07-30 02:45:21,617 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2022-09-16
2024-07-30 02:45:21,626 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2022-09-30
2024-07-30 02:45:21,635 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2022-10-14
2024-07-30 02:45:21,651 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2022-10-28
2024-07-30 02:45:21,670 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2022-11-11
2024-07-30 02:45:21,679 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2022-11-28
2024-07-30 02:45:21,688 - Interday Pairs Mean Reversion - INFO - Price series is not coi

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Interday Pairs Mean Reversion,^SPX,^IXIC
Measure,,,
Cumulative Return,2.994664,1.138128,1.096324
Annualized Return,0.633025,0.066770,0.064888
Annualized Volatility,0.771085,0.179564,0.240305
Annualized Sharpe Ratio,0.776013,0.178856,0.125817
Maximum Drawdown,-0.277689,-0.254251,-0.354928


# Explore cointegrated ETF portfolios to trade

In [30]:
start_date = datetime(2021,1,1)
end_date = get_today(-1)
n = 2

In [31]:
keys = ['commodity_type']

df_etf = get_all_etf()
df_etf = df_etf[~df_etf['name'].str.contains('short', case=False)]

df_etf = df_etf[df_etf['asset_class'] == 'Commodity']
symbols = list(df_etf['symbol'].unique())
df_etf_group = df_etf.groupby(keys)

ports = []
for key, df in df_etf_group:
    combs = df['symbol'].unique()
    for c in combinations(combs, n):
        ports.append((key, c))

In [34]:
strategy = InterdayMeanReversion('ETF Commod Mean Reversion')
strategy.set_capital(100000)
strategy.set_portfolio_comb(ports)
strategy.set_start_date(start_date)
strategy.set_end_date(end_date)
strategy.preprocess_data()
strategy.generate_position()
strategy.backtest_summary()

[*********************100%***********************]  78 of 78 completed


2024-07-29 01:59:02,298 - ETF Commod Mean Reversion - INFO - 727 of possible combination of portfolio over total 78 stocks
  0%|          | 0/727 [00:00<?, ?it/s]2024-07-29 01:59:02,400 - Interday Pairs Mean Reversion - INFO - Rebalance portfolio based on Johanson Test at rank 2: 2022-01-03
2024-07-29 01:59:02,412 - Interday Pairs Mean Reversion - INFO - Rebalance portfolio based on Johanson Test at rank 2: 2022-01-18
2024-07-29 01:59:02,425 - Interday Pairs Mean Reversion - INFO - Rebalance portfolio based on Johanson Test at rank 2: 2022-02-01
2024-07-29 01:59:02,459 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2022-02-28
2024-07-29 01:59:02,504 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2022-03-14
2024-07-29 01:59:02,528 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 202

[*********************100%***********************]  1 of 1 completed

2024-07-29 01:59:03,663 - Interday Pairs Mean Reversion - INFO - Rebalance portfolio based on Johanson Test at rank 2: 2022-01-03
2024-07-29 01:59:03,673 - Interday Pairs Mean Reversion - INFO - Rebalance portfolio based on Johanson Test at rank 2: 2022-01-18
2024-07-29 01:59:03,682 - Interday Pairs Mean Reversion - INFO - Rebalance portfolio based on Johanson Test at rank 2: 2022-02-01
2024-07-29 01:59:03,746 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2022-06-17
2024-07-29 01:59:03,755 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2022-07-05
2024-07-29 01:59:03,764 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2022-07-19
2024-07-29 01:59:03,772 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2022-08-02
20

2024-07-29 01:59:03,860 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2022-12-22
2024-07-29 01:59:03,869 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2023-01-09
2024-07-29 01:59:03,879 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2023-01-24
2024-07-29 01:59:03,887 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2023-02-07
2024-07-29 01:59:03,896 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2023-02-22
2024-07-29 01:59:03,905 - Interday Pairs Mean Reversion - INFO - Price series is not cointegrated, do not trade and refresh the eigenvector: 2023-03-08
2024-07-29 01:59:03,914 - Interday Pairs Mean Reversion - INFO - Price series is not coi

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,ETF Commod Mean Reversion,^SPX,^IXIC
Measure,,,
Cumulative Return,13.955437,1.138128,1.096324
Annualized Return,-24.846641,0.066770,0.064888
Annualized Volatility,25.298242,0.179564,0.240305
Annualized Sharpe Ratio,-0.983521,0.178563,0.125598
Maximum Drawdown,-3.555293,-0.254251,-0.354928


In [220]:
strategy.df_stats

,symbols,key,cumulative_return,annualized_return,annualized_volatility,annualized_sharpe_ratio,maximum_drawdown,actual_trade,portfolio_position,rank,is_mean_revert,rolling_not_mean_revert,pv_half_life,pv_hurst,pv_zscore
65,"GCC,SDCI","(Diversified,)",1.590319,0.245031,0.143382,1.463447,-0.075623,True,-1.500000,1,True,0,26.878566,0.484440,2.216972
69,"PDBC,UCIB","(Diversified,)",1.251982,0.115757,0.063947,1.259775,-0.034720,True,0.000000,1,False,0,16.121812,0.426907,0.308141
52,"DJCB,DJP","(Diversified,)",1.210679,0.098085,0.051762,1.214910,-0.029387,True,0.000000,1,False,0,0.717310,0.468093,-0.143261
76,"UCO,UGA","(Energy,)",1.581226,0.261973,0.246665,0.919363,-0.198398,False,0.000000,1,True,31,24.146652,0.570855,-0.275943
60,"FTGC,SDCI","(Diversified,)",3.094804,0.834710,0.875800,0.912892,-0.277667,False,-1.500000,1,True,0,29.133318,0.499160,2.594263
54,"DJCB,PDBC","(Diversified,)",1.232759,0.109411,0.083723,0.886404,-0.055164,False,0.000000,1,False,0,16.412701,0.490960,-0.675634
176,"PALL,SGOL","(Precious Metals,)",2.204031,0.513572,0.541483,0.883452,-0.259074,False,0.000000,1,True,61,57.386808,0.554100,-0.207251
119,"DBP,IAUF","(Precious Metals,)",1.892913,0.415678,0.444599,0.855781,-0.338243,False,0.558146,2,True,0,10.650481,0.399179,-0.013949
164,"IAUF,PPLT","(Precious Metals,)",2.157720,0.787243,0.922844,0.814921,-0.691649,False,-0.852349,2,True,36,65.355460,0.591710,0.112010
49,"DBC,GCC","(Diversified,)",1.178061,0.084797,0.061047,0.812474,-0.067671,False,0.511009,2,True,0,9.458886,0.380258,-1.044038
